# Coursera IBM Data Science Professional Qualification - Week 2 (Part 3)

``` "Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you." ```

In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import requests
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from IPython.core.display import display,HTML

## Load data from previous part of the assignment

In [2]:
filepath = 'week_2_part2_data.parquet'
toronto_df = pq.read_table(filepath).to_pandas()
toronto_df.head()

C:\Users\rosst\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\rosst\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\rosst\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


,Postal Code,Borough,Neighbourhood,postal_code,place_name,latitude,longitude,accuracy
0,M3A,North York,Parkwoods,M3A,North York (York Heights / Victoria Village / ...,43.7545,-79.3300,1.0
1,M4A,North York,Victoria Village,M4A,North York (Sweeney Park / Wigmore Park),43.7276,-79.3148,6.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,Downtown Toronto (Regent Park / Port of Toronto),43.6555,-79.3626,6.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,North York (Lawrence Manor / Lawrence Heights),43.7223,-79.4504,6.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,Queen's Park Ontario Provincial Government,43.6641,-79.3889,NaN


## Use K-means Clustering to Identify Areas with Similar Restaurants

### Define Functions to Query Foursquare:

Load credentials from file not uploaded to github:

In [3]:
s=pd.read_json('./secret/Foursquare_credentials.json')
CLIENT_ID = s.loc[0,'CLIENT_ID']
CLIENT_SECRET = s.loc[0,'CLIENT_SECRET']

Define a function to query foursquare, based on the API reference here: https://developer.foursquare.com/docs/api-reference/venues/search/#parameters

In [4]:
def foursquare_places_query(CLIENT_ID,CLIENT_SECRET, v,**kwargs):
    '''Returns the results of a foursquare query, based on the developer API, here: https://api.foursquare.com/v2/venues/search
    
    args:
    Required:
    CLIENT_ID
    CLIENT_SECRET
    v = version date in yyyymmdd format e.g. '20180605'
    
    Conditionally required (i.e. required if):
    The following are possible key word arguments & take string values (info from foursquare API reference https://developer.foursquare.com/docs/api-reference/venues/search/#parameters):
    
    ll = longitude latitude, required unless near is provided. Latitude and longitude of the user’s location specified as two numbers separated by a comma e.g. '1.0,1.0'
    near = required unless ll is provided. A string naming a place in the world. If the near string is not geocodable, returns a failed_geocode error. Otherwise, searches within the bounds of the geocode and adds a geocode object to the response.
    
    Optional:
    radius = Limit results to venues within this many meters of the specified location. Defaults to a city-wide area. Only valid for requests that use categoryId or query. 
    query = search term to be applied against venue names.
    limit = number of results to return, up to 50.
    categoryId = A comma separated list of categories to limit results to. If you specify categoryId. Specifying a radius may improve results. If specifying a top-level category, all sub-categories will also match the query.
    llAcc = Accuracy of latitude and longitude, in meters.
    alt = Altitude of the user’s location, in meters.
    altAcc = Accuracy of the user’s altitude, in meters.
    url = A third-party URL which foursquare will attempt to match against our map of venues to URLs.
    providerId = Identifier for a known third party that is part of our map of venues to URLs, used in conjunction with linkedId.
    linkedId = Identifier used by third party specified in providerId, which we will attempt to match against our map of venues to URLs.   
    '''
    base_url = 'https://api.foursquare.com/v2/venues/search?'
    query_string = '&client_id='+str(CLIENT_ID)+'&client_secret='+str(CLIENT_SECRET)+'&v='+str(v)
    #Construct the query string from the key word arguments:
    for key,value in kwargs.items():
        query_string = query_string+'&{}={}'.format(key,value)
    combined_url = base_url + query_string
    #Request data
    r =requests.get(combined_url).json()
    return r
    

Define a function to convert the foursquare data into a dataframe

In [5]:
def dataframe_of_foursquare_venues(CLIENT_ID,CLIENT_SECRET,v,**kwargs):
    '''Returns a dataframe of venues matching a foursquare query specified by the kwargs (see foursquare_places_query for detail on suitable kwargs)'''
    #Query foursquare
    foursquare_query = foursquare_places_query(CLIENT_ID,CLIENT_SECRET,v,**kwargs)
    #The results are in a json format, so convert this to a dataframe using json_normalize
    df = json_normalize(foursquare_query['response']['venues'])
    df.set_index(['id'])
    #Split out the venue categories
    def df_of_venue_categories(df):
        categories_df = pd.DataFrame()
        for index, row in df.iterrows():
            #isolate categories data for this venue
            temp_df = json_normalize(row['categories'])
            temp_df['id'] = row['id']
            temp_df.set_index('id')
            #Add the unfolded data to a consolidated dataframe
            if len(categories_df) == 0:
                categories_df = temp_df
            else:
                categories_df = categories_df.merge(temp_df,how='outer').fillna("")
        return categories_df
    df = df.join(df_of_venue_categories(df),how='left',rsuffix='.category')
    df.drop(columns=['categories'],inplace=True)
    return df
    

### For each neighbourhood in the Toronto Dataframe, use the Foursquare API to find out what the top restaurants are

In [6]:
FOURSQUARE_VERSION = '20180605'

In [7]:
for index, row in toronto_df.iterrows():
    longlatstr = str(row['latitude'])+','+str(row['longitude'])
    try:
        #load the restaurant data for this neighbourhood
        restaurant_df = dataframe_of_foursquare_venues(CLIENT_ID,CLIENT_SECRET,v=FOURSQUARE_VERSION,ll=longlatstr,query='restaurant')
        #count the top restaurants by shortName e.g. 'Indian'
        count_of_top_restaurants  = restaurant_df['shortName'].value_counts()
        #output this information into toronto_df & report it to the user
        for key, value in count_of_top_restaurants.to_dict().items():
            #if a suitable column exists then add the value, otherwise add a new column & populate that
            if key in toronto_df.columns:
                toronto_df.loc[index,key]= value
            else:
                #Initialise the new column with zeros
                toronto_df[key] = 0
                toronto_df.loc[index,key] = value
        print('Top {} Restaurants in {} are: \n {}'.format(sum(count_of_top_restaurants),row['Neighbourhood'],count_of_top_restaurants))
    except:
        print('Could not download venues for {} '.format(row['Neighbourhood']))

Top 30 Restaurants in Parkwoods are: 
 Chinese           9
Caribbean         4
Japanese          4
Indian            2
Vietnamese        2
Cantonese         2
Middle Eastern    1
Italian           1
Szechuan          1
Xinjiang          1
Korean            1
Restaurant        1
Asian             1
Name: shortName, dtype: int64
Top 30 Restaurants in Victoria Village are: 
 Chinese          5
Caribbean        5
Japanese         4
Indian           3
Restaurant       3
Breakfast        1
Vietnamese       1
Xinjiang         1
Italian          1
Ethiopian        1
Korean           1
Asian            1
Mediterranean    1
Filipino         1
Thai             1
Name: shortName, dtype: int64
Top 30 Restaurants in Regent Park, Harbourfront are: 
 Chinese        5
Restaurant     5
American       4
Bar            2
Breakfast      2
Event Space    1
Indian         1
Noodles        1
Korean         1
Dim Sum        1
Ethiopian      1
Wine Bar       1
Szechuan       1
Italian        1
Vietnamese     1


In [8]:
#drop irrelevant columns, na values and restaurants with category 'restaurant' as this is not a useful description for the purpose of this assignment
try:
    toronto_df.drop(columns=['postal_code', 'place_name', 'accuracy'],inplace=True)
except:
    pass
try:
     toronto_df.drop(columns=['Restaurant'],inplace=True)
except:
    pass

toronto_df.dropna(inplace=True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood,latitude,longitude,Chinese,Caribbean,Japanese,Indian,Vietnamese,...,Afghan,Halal,Fried Chicken,Dumplings,Other Event,Warehouse Store,Music Venue,Furniture / Home,Dutch,Polish
0,M3A,North York,Parkwoods,43.7545,-79.3300,9,4,4,2,2,...,0,0,0,0,0,0,0,0,0,0
1,M4A,North York,Victoria Village,43.7276,-79.3148,5,5,4,3,1,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,5,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,5,1,1,1,2,...,0,0,0,0,0,0,0,0,0,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,5,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Incase rerunning part of the code, remove cluster labels
try:
    toronto_df.drop(columns=['Cluster Labels'],inplace=True)
    print('Old Cluster Labels Removed, ready for rerun')
except:
    pass

## Apply Clustering Algorithm to identify similar areas for restaurant options/prefererences

### Apply k-means clusteringto identify similar neighbourhoods & show these on a map

In [10]:
#Before applying kmeans clustering to toronto_df, remove the string columns which do not relate to venue types
toronto_df_clean = toronto_df.drop(columns=['Postal Code','Borough','Neighbourhood','latitude','longitude'])
toronto_df_clean.head()

,Chinese,Caribbean,Japanese,Indian,Vietnamese,Cantonese,Middle Eastern,Italian,Szechuan,Xinjiang,...,Afghan,Halal,Fried Chicken,Dumplings,Other Event,Warehouse Store,Music Venue,Furniture / Home,Dutch,Polish
0,9,4,4,2,2,2,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,5,5,4,3,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,5,1,1,1,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#set number of clusters
kclusters = 7


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_df_clean)

# check cluster labels generated for each row in the dataframe
print('Kmeans labels are {}'.format(kmeans.labels_))

Kmeans labels are [5 4 2 6 2 1 3 5 4 2 1 0 3 4 4 2 1 0 3 4 2 1 3 4 2 1 3 3 5 4 2 1 3 5 5 0 2
 1 3 5 6 4 2 1 4 5 6 4 2 1 5 5 6 6 2 6 1 6 4 5 5 6 1 1 6 3 5 4 1 1 6 5 3 1
 1 1 6 3 6 1 1 3 1 2 3 1 2 2 0 3 6 2 0 0 3 2 2 1 2 3 1 6]


In [12]:
toronto_df.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_df.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,latitude,longitude,Chinese,Caribbean,Japanese,Indian,...,Afghan,Halal,Fried Chicken,Dumplings,Other Event,Warehouse Store,Music Venue,Furniture / Home,Dutch,Polish
0,5,M3A,North York,Parkwoods,43.7545,-79.3300,9,4,4,2,...,0,0,0,0,0,0,0,0,0,0
1,4,M4A,North York,Victoria Village,43.7276,-79.3148,5,5,4,3,...,0,0,0,0,0,0,0,0,0,0
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,5,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,5,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,5,1,0,1,...,0,0,0,0,0,0,0,0,0,0


### Map the clusters

In [13]:
# create map
map_clusters = folium.Map(location=[toronto_df['latitude'].mean(), toronto_df['longitude'].mean()], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#Add each cluster to a list, its index will determine the color displayed
clusters = []

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_df['latitude'], toronto_df['longitude'], toronto_df['Neighbourhood'], toronto_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #Work out the index to use for coloring
    if cluster not in clusters:
        clusters.append(cluster)
    cluster_index = clusters.index(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster_index],
        color=colors.rgb2hex(colors_array[cluster_index]),
        fill=True,
        fill_color=rainbow[cluster_index],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Look at what makes up the clusters:

In [14]:
for cluster_num in range(kclusters):
    #Restrict to just the datadrame containing the current cluster label:
    cluster_df = toronto_df[toronto_df['Cluster Labels']==cluster_num]
    #remove long and lat as these aren't needed for this analysis
    cluster_df.drop(columns=['longitude','latitude','Cluster Labels'],inplace=True)
    #work out which are the most common entries by taking an average across all items in the cluster
    frequency_of_category = cluster_df.mean().sort_values(ascending=False)
    #Add a title
    display(HTML("<h3>Restaurants in Cluster sorted from most to least popular {} </h3>".format(cluster_num)))
    #Show the list in order of most to least frequent
    display(frequency_of_category.index)


C:\Users\rosst\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Index(['Chinese', 'American', 'Ethiopian', 'Caribbean', 'Vietnamese',
       'Middle Eastern', 'Korean', 'Asian', 'Indian', 'Italian', 'Xinjiang',
       'Iraqi Restaurant', 'Latin American', 'Thai', 'Diner', 'Turkish',
       'Breakfast', 'Nightclub', 'New American', 'African', 'Japanese',
       'Pizza', 'Greek', 'Cantonese', 'Scandinavian', 'Mexican', 'Steakhouse',
       'Hong Kong', 'Dim Sum', 'Shop', 'Lounge', 'Noodles', 'Sushi',
       'Wine Bar', 'Hakka', 'Event Space', 'Bar', 'Filipino', 'Mediterranean',
       'Cha Chaan Teng', 'Szechuan', 'German', 'Food & Drink',
       'Vegetarian / Vegan', 'Indian Chinese', 'Polish', 'Dive Bar',
       'Moroccan', 'Seafood', 'Hunan', 'Peruvian', 'Deli / Bodega', 'French',
       'Juice Bar', 'Nightlife', 'Afghan', 'Karaoke', 'Halal', 'Fried Chicken',
       'Dumplings', 'Other Event', 'Warehouse Store', 'Music Venue',
       'Furniture / Home', 'Caucasian', 'Tibetan', 'Burgers', 'Pub',
       'Grocery Store', 'Dutch', 'Vineyard', 'Hungari

Index(['Korean', 'Chinese', 'New American', 'Ethiopian', 'Vietnamese',
       'Dim Sum', 'Nightclub', 'Noodles', 'Diner', 'Thai', 'Japanese',
       'Breakfast', 'Caribbean', 'Italian', 'Bar', 'American', 'Event Space',
       'Indian', 'Latin American', 'Turkish', 'Middle Eastern', 'Portuguese',
       'Asian', 'Café', 'Szechuan', 'Sushi', 'Xinjiang', 'Deli / Bodega',
       'Cantonese', 'Steakhouse', 'Eastern European', 'Mexican', 'French',
       'Iraqi Restaurant', 'Pizza', 'Grocery Store', 'Filipino',
       'Mediterranean', 'Polish', 'Argentinian', 'Nightlife', 'Tibetan',
       'Karaoke', 'Sandwiches', 'Pub', 'Mac & Cheese', 'Spanish', 'Juice Bar',
       'Afghan', 'Moroccan', 'Halal', 'Lounge', 'Peruvian', 'Fried Chicken',
       'Dumplings', 'Hunan', 'Other Event', 'Seafood', 'Warehouse Store',
       'Caucasian', 'Music Venue', 'Furniture / Home', 'Persian', 'Lebanese',
       'Hungarian', 'Shop', 'Greek', 'Vineyard', 'Dutch',
       'Molecular Gastronomy', 'Burgers', 'Dive B

Index(['Chinese', 'American', 'Bar', 'Breakfast', 'Dim Sum', 'Caribbean',
       'Indian', 'Korean', 'Noodles', 'Thai', 'New American', 'Szechuan',
       'Wine Bar', 'Vietnamese', 'Nightclub', 'Event Space', 'Diner',
       'Italian', 'Asian', 'Ethiopian', 'Japanese', 'Sushi', 'Filipino',
       'Molecular Gastronomy', 'Pub', 'Spanish', 'Café', 'African', 'Dutch',
       'Xinjiang', 'Mongolian', 'Greek', 'Iraqi Restaurant',
       'Vegetarian / Vegan', 'Food & Drink', 'German', 'Cha Chaan Teng',
       'Cantonese', 'Hong Kong', 'Mediterranean', 'Middle Eastern', 'Turkish',
       'Pizza', 'Latin American', 'Hakka', 'Indian Chinese', 'Polish',
       'Dive Bar', 'Moroccan', 'Seafood', 'Hunan', 'Peruvian', 'Deli / Bodega',
       'French', 'Juice Bar', 'Nightlife', 'Afghan', 'Burgers', 'Halal',
       'Fried Chicken', 'Dumplings', 'Other Event', 'Warehouse Store',
       'Music Venue', 'Furniture / Home', 'Caucasian', 'Karaoke', 'Tibetan',
       'Persian', 'Grocery Store', 'Vineyard', 

Index(['Chinese', 'Korean', 'Japanese', 'Indian', 'Caribbean',
       'Vegetarian / Vegan', 'Hong Kong', 'Breakfast', 'Asian', 'Hakka',
       'Xinjiang', 'Cantonese', 'German', 'Vietnamese', 'Cha Chaan Teng',
       'Dumplings', 'Warehouse Store', 'Indian Chinese', 'American', 'Italian',
       'Szechuan', 'Filipino', 'Other Event', 'Middle Eastern', 'Sushi',
       'Dim Sum', 'Latin American', 'Furniture / Home', 'Food & Drink',
       'Turkish', 'Pizza', 'Pub', 'Food Court', 'Steakhouse',
       'Eastern European', 'French', 'Thai', 'Bar', 'African', 'Ethiopian',
       'Wine Bar', 'New American', 'Noodles', 'Nightclub', 'Diner',
       'Iraqi Restaurant', 'Event Space', 'Mediterranean', 'Polish',
       'Dive Bar', 'Burgers', 'Persian', 'Tibetan', 'Karaoke', 'Caucasian',
       'Seafood', 'Hunan', 'Peruvian', 'Deli / Bodega', 'Juice Bar',
       'Nightlife', 'Moroccan', 'Afghan', 'Halal', 'Fried Chicken',
       'Music Venue', 'Lebanese', 'Lounge', 'Café', 'Spanish', 'Grocery Store

Index(['Caribbean', 'Chinese', 'Japanese', 'Indian', 'Italian', 'Thai',
       'Ethiopian', 'Korean', 'Asian', 'Vietnamese', 'American', 'Filipino',
       'Breakfast', 'Greek', 'Event Space', 'New American', 'Hungarian',
       'Diner', 'German', 'African', 'Burgers', 'Nightclub', 'Sushi',
       'Noodles', 'Dive Bar', 'Xinjiang', 'Mediterranean', 'Bar', 'Turkish',
       'Vineyard', 'Middle Eastern', 'Szechuan', 'Dim Sum', 'Persian',
       'Lebanese', 'Shop', 'Wine Bar', 'Hong Kong', 'Cha Chaan Teng', 'Pizza',
       'Latin American', 'Indian Chinese', 'Food & Drink',
       'Vegetarian / Vegan', 'Cantonese', 'Iraqi Restaurant', 'Polish',
       'Hakka', 'Moroccan', 'Seafood', 'Hunan', 'Peruvian', 'Deli / Bodega',
       'French', 'Juice Bar', 'Nightlife', 'Afghan', 'Grocery Store', 'Halal',
       'Fried Chicken', 'Dumplings', 'Other Event', 'Warehouse Store',
       'Music Venue', 'Furniture / Home', 'Caucasian', 'Karaoke', 'Tibetan',
       'Lounge', 'Dutch', 'Molecular Gastronom

Index(['Chinese', 'Japanese', 'Caribbean', 'Korean', 'Middle Eastern',
       'Vietnamese', 'Indian', 'Hong Kong', 'Cantonese', 'Szechuan', 'Pizza',
       'Xinjiang', 'Steakhouse', 'Asian', 'Italian', 'Turkish', 'Breakfast',
       'Latin American', 'Diner', 'Ethiopian', 'American', 'Seafood',
       'Caucasian', 'Sushi', 'Hakka', 'Cha Chaan Teng', 'Bar', 'Wings', 'Café',
       'Thai', 'German', 'Fried Chicken', 'Halal', 'Afghan', 'Moroccan',
       'Peruvian', 'Filipino', 'Hunan', 'Eastern European',
       'Vegetarian / Vegan', 'Mediterranean', 'African', 'New American',
       'Wine Bar', 'Nightclub', 'Dim Sum', 'Iraqi Restaurant', 'Noodles',
       'Event Space', 'Polish', 'Food & Drink', 'French', 'Lounge', 'Lebanese',
       'Persian', 'Tibetan', 'Karaoke', 'Deli / Bodega', 'Juice Bar', 'Shop',
       'Nightlife', 'Dumplings', 'Other Event', 'Warehouse Store',
       'Music Venue', 'Furniture / Home', 'Portuguese', 'Food Court',
       'Indian Chinese', 'Hungarian', 'Dive Bar',

Index(['Chinese', 'Korean', 'Ethiopian', 'Vietnamese', 'Caribbean', 'Japanese',
       'Nightclub', 'Middle Eastern', 'American', 'Thai', 'Latin American',
       'Italian', 'Turkish', 'Indian', 'Pizza', 'Hong Kong', 'New American',
       'Steakhouse', 'Noodles', 'Event Space', 'Xinjiang', 'Breakfast',
       'Szechuan', 'African', 'Diner', 'Asian', 'Dim Sum', 'Filipino',
       'Iraqi Restaurant', 'Sushi', 'Bar', 'Mediterranean', 'Seafood',
       'French', 'Music Venue', 'Wine Bar', 'Dive Bar', 'Cantonese',
       'Cha Chaan Teng', 'German', 'Food & Drink', 'Vegetarian / Vegan',
       'Indian Chinese', 'Hakka', 'Polish', 'Burgers', 'Grocery Store',
       'Persian', 'Tibetan', 'Karaoke', 'Caucasian', 'Hunan', 'Peruvian',
       'Deli / Bodega', 'Juice Bar', 'Nightlife', 'Moroccan', 'Afghan',
       'Halal', 'Fried Chicken', 'Dumplings', 'Other Event', 'Warehouse Store',
       'Furniture / Home', 'Lebanese', 'Lounge', 'Café', 'Mexican', 'Dutch',
       'Vineyard', 'Greek', 'Hungari